In [1]:
from pathlib import Path
from typing import Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
from torchvision import transforms

from PIL import Image
from tqdm import tqdm

from utils import locate_data

In [2]:
import os

In [3]:
image_transforms = ["flip",
                    "crop",
                    "perspective",
                    "jitter"]

In [4]:
data_path = Path("data/CamVid")

In [5]:
image_paths = sorted((data_path / "train").glob("*"))
mask_paths = sorted((data_path / "train_labels").glob("*"))

In [6]:
torch.cuda.is_available()

True

In [7]:
for t in image_transforms:
    os.mkdir(data_path / ("train_"+ t))
    os.mkdir(data_path / ("train_labels_"+ t))

FileExistsError: [Errno 17] File exists: 'data/CamVid/train_flip'

In [8]:
flip_transform = transforms.RandomHorizontalFlip(p=1)
crop_transform = transforms.RandomCrop(size=(360, 480))
perspective_transform = transforms.Compose(
            [transforms.RandomPerspective(distortion_scale=0.5, p=1),
            transforms.CenterCrop(size=(360, 480))])
jitter_transform = transforms.ColorJitter(brightness=(1.2,2),hue=(-0.5,0.5))

tensor_transform = transforms.ToTensor()




flip_images = list()
crop_images = list()
perspective_images = list()
jitter_images = list()

flip_masks = list()
crop_masks = list()
perspective_masks = list()
jitter_masks = list()



for i in tqdm(range(len(image_paths))):
    image_path = image_paths[i]
    mask_path = mask_paths[i]

    with open(image_path, "rb") as image_file, open(mask_path, "rb") as mask_file:
        image = Image.open(image_file)
        mask = Image.open(mask_file)

        image = tensor_transform(image)
        mask =  tensor_transform(mask)


        flip_images.append(flip_transform(image))
        flip_masks.append(flip_transform(mask))

        crop_images.append(crop_transform(image))
        crop_masks.append(crop_transform(mask))

        perspective_images.append(perspective_transform(image))
        perspective_masks.append(perspective_transform(mask))

        jitter_images.append(jitter_transform(image))
        jitter_masks.append(jitter_transform(mask))
        
        

100%|██████████| 369/369 [01:53<00:00,  3.25it/s]


In [10]:
toimage = transforms.ToPILImage()

In [15]:
os.path.basename(image_paths[0])

'0001TP_009210.png'

In [18]:
for i in tqdm(range(len(image_paths))):


    image_name = os.path.basename(image_paths[i])
    mask_name = os.path.basename(mask_paths[i])

    toimage(flip_images[i]).save(data_path /("train_flip/"+ image_name))
    toimage(flip_masks[i]).save(data_path /("train_labels_flip/"+ mask_name))
    toimage(crop_images[i]).save(data_path /("train_crop/"+ image_name))
    toimage(crop_masks[i]).save(data_path /("train_labels_crop/"+ mask_name))
    toimage(perspective_images[i]).save(data_path /("train_perspective/"+ image_name))
    toimage(perspective_masks[i]).save(data_path /("train_labels_perspective/"+ mask_name))
    toimage(jitter_images[i]).save(data_path /("train_jitter/"+ image_name))
    toimage(jitter_masks[i]).save(data_path /("train_labels_jitter/"+ mask_name))


100%|██████████| 369/369 [04:12<00:00,  1.46it/s]


In [17]:
data_path /("train_flip/"+ os.path.basename(image_paths[0]))

PosixPath('data/CamVid/train_flip/0001TP_009210.png')

In [19]:
from segmentationData import SegmentationDataset

In [ ]:
transformations = transforms.Compose(
                              [
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                            ]
                            )


dataset = SegmentationDataset(Path("data/CamVid"),
                              transforms=transformations,
                              image_folder="train",
                              mask_folder="train_labels",
                              augments=["crop"]
                             )